In [1]:
!pip install youtube-dl moviepy
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.8 kB/s eta 0:00:0000:0100:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 26.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 25.4 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 33.0 kB/s eta 0:00:0000:0100:22
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 46.6 kB/s eta 0:00:0000:0100:02
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=1e4d488d2798c25f49c8385c1319bda6afc7487ef8d0c39449d226b5e204eb02
  Stored in directory: /Users/afrozanowshin/Library/Caches/pip/wheels/83/b1/d9/119ef7c144b44d591ec0a9a140465133c23ea95d2a161184ba
Successfully built moviepy
  Attempting uninstall: pillow
    Found existing installation: Pillow 10.1.0
    Uninstalling Pillow-10.1.0:
      Successfully uninstalled Pillow-10.1.0
  Attempting uninstall: decorator
    Found exi

In [4]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

## **<font style="color:rgb(134,19,348)">Preprocess and Create the Dataset</font>**

In [6]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 100, 100

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 30

# Specify the directory containing the UCF50 dataset.
DATASET_DIR = "J:/Medi_Resized_224"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ["Asthma", "Bandage", "Blood", "Blood Pressure", "Broke", "Burn", "Cold", "Constipated", "Cut", "Diarrhea", "Disease", "Doctor", "Emergency", "Fever", "Headache", "Infection", "Itch", "Nauseous", "Pain", "Patient", "Redness", "Sore Throat", "Urgent Care (UC)", "Anxiety", "Depressed", "Hospital", "Sprain", "Swallow", "Treatment", "Vomit"]

In [7]:
def frames_extraction(video_path):
    '''
    This function will extract the required frames from a video after resizing and normalizing them.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
    Returns:
        frames_list: A list containing the resized and normalized frames of the video.
    '''

    # Declare a list to store video frames.
    frames_list = []

    # Read the Video File using the VideoCapture object.
    video_reader = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    # Iterate through the Video Frames.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Reading the frame from the video.
        success, frame = video_reader.read()

        # Check if Video frame is not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Append the normalized frame into the frames list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object.
    video_reader.release()

    # Return the frames list.
    return frames_list

In [8]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []

    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):

        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))

        # Iterate through all the files present in the files list.
        for file_name in files_list:

            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)

            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:

                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)

    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [ ]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

In [ ]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.2, shuffle = True,
                                                                            random_state = seed_constant)

## **<font style="color:rgb(134,19,348)">Create, Train, and Test the Mobilenet-based LRCN model</font>**

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed, Dropout, Flatten
from tensorflow.keras.models import Sequential

def create_LRCN_MOBmodel():
    '''
    This function will construct the required LRCN model with MobileNetV2 as the CNN feature extractor.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # Create the MobileNetV2 model with non-trainable weights
    mobilenetv2_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
    for layer in mobilenetv2_base.layers:
        layer.trainable = False

    # We will use a Sequential model for model construction.
    model = Sequential()

    # Define the Model Architecture.
    ########################################################################################################################

    # Use TimeDistributed layer to apply MobileNetV2 features to each frame in the sequence
    model.add(TimeDistributed(mobilenetv2_base, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    #model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Conv2D(64, (1, 1), padding='same', activation='relu')))
    #model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))

    model.add(TimeDistributed(Flatten()))
    lstm_fw = LSTM(128)
    lstm_bw = LSTM(units=128, go_backwards=True)
    model.add(Bidirectional(lstm_fw, backward_layer=lstm_bw))

    model.add(Dense(len(CLASSES_LIST), activation='softmax'))

    ########################################################################################################################

    # Display the model's summary.
    model.summary()

    # Return the constructed LRCN model.
    return model

In [ ]:
# Construct the required LRCN model.
LRCN_Mobimodel = create_LRCN_MOBmodel()

# Display the success message.
print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 30, 4, 4, 1280)    2257984   
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 30, 4, 4, 1280)    0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 30, 4, 4, 64)      81984     
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 30, 4, 4, 64)      0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 30, 1024)          0

In [ ]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_Mobimodel.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start training the model.
LRCN_MOBImodel_training_history = LRCN_Mobimodel.fit(x = features_train, y = labels_train, epochs = 100, batch_size = 16,
                                             shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/100
144/144 [==============================] - 201s 1s/step - loss: 3.4651 - accuracy: 0.0414 - val_loss: 3.3923 - val_accuracy: 0.0522
Epoch 2/100
144/144 [==============================] - 233s 2s/step - loss: 3.3564 - accuracy: 0.0640 - val_loss: 3.3157 - val_accuracy: 0.0783
Epoch 3/100
144/144 [==============================] - 230s 2s/step - loss: 3.1415 - accuracy: 0.1206 - val_loss: 2.9600 - val_accuracy: 0.1443
Epoch 4/100
144/144 [==============================] - 230s 2s/step - loss: 2.6157 - accuracy: 0.2660 - val_loss: 2.4222 - val_accuracy: 0.2852
Epoch 5/100
144/144 [==============================] - 229s 2s/step - loss: 2.0643 - accuracy: 0.4145 - val_loss: 1.9644 - val_accuracy: 0.4017
Epoch 6/100
144/144 [==============================] - 229s 2s/step - loss: 1.5811 - accuracy: 0.5620 - val_loss: 1.5332 - val_accuracy: 0.5287
Epoch 7/100
144/144 [==============================] - 229s 2s/step - loss: 1.1592 - accuracy: 0.6870 - val_loss: 1.2526 - val_accuracy:

In [ ]:
model_evaluation_history = LRCN_Mobimodel.evaluate(features_test, labels_test)

23/23 [==============================] - 36s 2s/step - loss: 0.3104 - accuracy: 0.9332


In [ ]:
#Print classification report, confusion matrix, F1 score, Recall, Precision
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score
print(classification_report(y_true=labels_test.argmax(axis=1), y_pred=LRCN_Mobimodel.predict(features_test).argmax(axis=1)))
print(confusion_matrix(y_true=labels_test.argmax(axis=1), y_pred=LRCN_Mobimodel.predict(features_test).argmax(axis=1)))
print("F1 score: ", f1_score(y_true=labels_test.argmax(axis=1), y_pred=LRCN_Mobimodel.predict(features_test).argmax(axis=1), average="macro"))
print("Recall: ", recall_score(y_true=labels_test.argmax(axis=1), y_pred=LRCN_Mobimodel.predict(features_test).argmax(axis=1), average="macro"))
print("Precision: ", precision_score(y_true=labels_test.argmax(axis=1), y_pred=LRCN_Mobimodel.predict(features_test).argmax(axis=1), average="macro"))

23/23 [==============================] - 39s 2s/step
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        24
           1       0.89      1.00      0.94        25
           2       0.97      1.00      0.98        29
           3       0.97      1.00      0.98        30
           4       0.91      0.71      0.80        28
           5       0.88      0.93      0.90        30
           6       0.94      0.94      0.94        18
           7       0.95      1.00      0.97        18
           8       0.95      0.88      0.91        24
           9       0.78      1.00      0.88        21
          10       0.95      1.00      0.98        21
          11       1.00      0.78      0.88        27
          12       0.91      1.00      0.95        21
          13       0.96      0.96      0.96        26
          14       1.00      1.00      1.00        30
          15       0.95      1.00      0.97        19
          16       1.00     

In [ ]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

# Define the string date format.
# Get the current Date and Time in a DateTime Object.
# Convert the DateTime object to string according to the style mentioned in date_time_format string.
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'Mobi_LRCN_model_LSTM_128_Date_Time_{current_date_time_string}___Loss_{model_evaluation_loss}___Accuracy_{model_evaluation_accuracy}.h5'

# Save the Model.
LRCN_Mobimodel.save(model_file_name)

## **<font style="color:rgb(134,19,348)">Prediction</font>**

In [ ]:
test_videos_directory = 'J:/Test Videos/'

In [ ]:
def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    '''
    This function will perform single action recognition prediction on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store video frames we will extract.
    frames_list = []

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read()

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = LRCN_Mobimodel.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]

    # Display the predicted action along with the prediction confidence.
    print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')

    # Release the VideoCapture object.
    return predicted_class_name
    video_reader.release()

In [ ]:
video_title = 'Patient'

# Construct tihe nput youtube video path
input_video_file_path = f'{test_videos_directory}/{video_title}.mp4'

# Perform Single Prediction on the Test Video.
predict_single_action(input_video_file_path, SEQUENCE_LENGTH)

# Display the input video.
VideoFileClip(input_video_file_path, audio=False, target_resolution=(300,None)).ipython_display()


1/1 [==============================] - 0s 95ms/step
Action Predicted: Patient
Confidence: 0.9999297857284546
Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
